In [13]:
# https://xilinx.github.io/brevitas/getting_started.html
# works with automated folding.

# LiDAR data rate is 10-30 frames per second. Considering 30 frames per second, resolution is 30Hz or 33.33ms.

from torch.nn import Module
import torch.nn.functional as F

import brevitas.nn as qnn
from brevitas.quant import Int8Bias as BiasQuant
from brevitas.quant import Int32Bias

import os
import onnx
import torch
import numpy as np
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, Dataset
from brevitas.nn import QuantLinear, QuantReLU
import torch.nn as nn
from sklearn.metrics import accuracy_score
from tqdm import tqdm, trange

weight_bit_width = 4
act_bit_width = 4

torch.manual_seed(0)

class QuantWeightActBiasLeNet(Module):
    def __init__(self):
        super(QuantWeightActBiasLeNet, self).__init__()
        self.quant_inp = qnn.QuantIdentity(bit_width=4, return_quant_tensor=True)
        self.conv1 = qnn.QuantConv2d(20, 6, 5, bias=False, weight_bit_width=weight_bit_width)
        self.relu1 = qnn.QuantReLU(bit_width=act_bit_width, return_quant_tensor=True)
        self.conv2 = qnn.QuantConv2d(6, 16, 5, bias=False, weight_bit_width=weight_bit_width)
        self.relu2 = qnn.QuantReLU(bit_width=act_bit_width, return_quant_tensor=True)
        self.flatten = nn.Flatten()
        self.fc1   = qnn.QuantLinear(16*4*4, 120, bias=False, weight_bit_width=weight_bit_width)
        # self.fc1   = qnn.QuantLinear(16*5*5, 120, bias=True, weight_bit_width=weight_bit_width, bias_quant=Int32Bias)
        self.relu3 = qnn.QuantReLU(bit_width=act_bit_width, return_quant_tensor=True)
        self.fc2   = qnn.QuantLinear(120, 84, bias=False, weight_bit_width=weight_bit_width)
        self.relu4 = qnn.QuantReLU(bit_width=act_bit_width, return_quant_tensor=True)
        self.fc3   = qnn.QuantLinear(84, 64, bias=False, weight_bit_width=weight_bit_width)

    def forward(self, x):
        out = self.quant_inp(x)
        out = self.relu1(self.conv1(out))
        out = F.max_pool2d(out, 2)
        out = self.relu2(self.conv2(out))
        # out = F.max_pool2d(out, 2)
        # out = out.reshape(out.shape[0], -1)
        out = self.flatten(out)
        out = self.relu3(self.fc1(out))
        out = self.relu4(self.fc2(out))
        out = self.fc3(out)
        return out

    # def forward(self, x):
    #     out = self.quant_inp(x)
    #     out = self.relu1(self.conv1(out))
    #     print(f"Shape after conv1: {out.shape}")
    #     out = F.max_pool2d(out, 2)
    #     out = self.relu2(self.conv2(out))
    #     print(f"Shape after conv2: {out.shape}")
    #     # out = F.max_pool2d(out, 2)
    #     out = self.quant_inp(out)
    #     # out = out.reshape(out.shape[0], -1)
    #     out = self.flatten(out)
    #     print(f"Shape after flattening: {out.shape}")
    #     out = self.relu3(self.fc1(out))
    #     print(f"Shape after fc1: {out.shape}")
    #     out = self.relu4(self.fc2(out))
    #     print(f"Shape after fc2: {out.shape}")
    #     out = self.fc3(out)
    #     return out

model = QuantWeightActBiasLeNet()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Target device: " + str(device))

model.to(device); # The semicolon is for not printing the model.


Target device: cpu


In [14]:
input_tensor = torch.randn(1, 20, 20, 20)
output = model(input_tensor)

### Data processing

In [15]:
# This is for 4-bit quantization. The quantization value can be changed in line 11. 

import numpy as np
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import os

# quant_param = 4 # log2(quant_param) bits. #256 means no quanization, 2 means 1-bit quantization

# def quantize_image(image):
#     """Quantize and binarize an image."""
#     image = image.astype(np.float32)
#     image = np.floor(image / (256/quant_param))  # Example: reducing to 4-bit quantization
#     return image.astype(np.float32)

# def save_dataset_as_npz(data, labels, filename):
#     """Save the dataset as a .npz file."""
#     np.savez_compressed(filename, data=data, labels=labels)

# # Load CIFAR-10 dataset
# transform = transforms.Compose([transforms.ToTensor()])
# train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
# test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# # Quantize the images
# train_images = np.array([quantize_image(image.numpy().transpose(1, 2, 0) * 255) for image, _ in train_dataset])
# train_labels = np.array(train_dataset.targets)

# test_images = np.array([quantize_image(image.numpy().transpose(1, 2, 0) * 255) for image, _ in test_dataset])
# test_labels = np.array(test_dataset.targets)

# # Save the datasets
# os.makedirs('./quantized_data', exist_ok=True)
# save_dataset_as_npz(train_images, train_labels, './quantized_data/cifar10_train.npz')
# save_dataset_as_npz(test_images, test_labels, './quantized_data/cifar10_test.npz')

# print('Saved the dataset as .npz files')


class CIFAR10QuantizedDataset(Dataset):
    def __init__(self, npz_file):
        data = np.load(npz_file)
        self.images = data['data']
        self.labels = data['label']

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx].astype(np.float32) / 255.0
        label = self.labels[idx]
        image = torch.tensor(image.transpose(2, 0, 1))  # HWC to CHW format
        label = torch.tensor(label, dtype=torch.long)
        return image, label

# Load the quantized dataset
train_dataset = CIFAR10QuantizedDataset('/home/inets/arish/FINN_arish/finn/notebooks/Testing_arish/data/X_lidar_train.npz')
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = CIFAR10QuantizedDataset('/home/inets/arish/FINN_arish/finn/notebooks/Testing_arish/data/X_lidar_test.npz')
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


### Training and testing

In [16]:
import torch.optim as optim

def train(model, train_loader, optimizer, criterion):
    losses = []
    # ensure model is in training mode
    model.train()    
    
    for i, data in enumerate(train_loader, 0):        
        images, labels = data
        labels = torch.argmax(labels, dim=1)
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()   
                
        # forward pass
        output = model(images.float())
        # loss = criterion(output, labels.unsqueeze(1))
        loss = criterion(output, labels)
        
        # backward pass + run optimizer to update weights
        loss.backward()
        optimizer.step()
        
        # keep track of loss value
        losses.append(loss.data.cpu().numpy()) 
    return losses


def test(model, test_loader):    
    # ensure model is in eval mode
    model.eval() 
    y_true = []
    y_pred = []

   
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            output = model(images.float())
            # run the output through sigmoid
            # output = torch.sigmoid(output_orig)  
            # compare against a threshold of 0.5 to generate 0/1
            # pred = (output.detach().cpu().numpy() > 0.5) * 1
            _, pred = torch.max(output.data, 1)
            # labels = torch.argmax(labels, dim=1) #
            labels = labels.cpu().float()  
            # Convert one-hot encoded labels to class indices
            if labels.ndim > 1 and labels.size(1) > 1:
                labels = torch.argmax(labels, dim=1)
            y_true.extend(labels.tolist()) 
            y_pred.extend(pred.reshape(-1).tolist())
            # y_pred.extend((pred == labels).sum().item())
        
    return accuracy_score(y_true, y_pred)

num_epochs = 10
lr = 0.001 

def display_loss_plot(losses, title="Training loss", xlabel="Iterations", ylabel="Loss"):
    x_axis = [i for i in range(len(losses))]
    plt.plot(x_axis,losses)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()

criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999))

# Setting seeds for reproducibility
torch.manual_seed(0)
np.random.seed(0)

running_loss = []
running_test_acc = []
t = trange(num_epochs, desc="Training loss", leave=True)

for epoch in t:
        loss_epoch = train(model, train_loader, optimizer,criterion)
        test_acc = test(model, test_loader)
        t.set_description("Training loss = %f test accuracy = %f" % (np.mean(loss_epoch), test_acc))
        t.refresh() # to show immediately the update           
        running_loss.append(loss_epoch)
        running_test_acc.append(test_acc)


Training loss = 2.405633 test accuracy = 0.323091: 100%|███████████████████████████████████████████████████████████████████████| 10/10 [01:04<00:00,  6.42s/it]


In [17]:
test_accuracy = test(model, test_loader)
print('test accuracy = ', test_accuracy)

test accuracy =  0.3230909644052327


In [6]:
# # Save the Brevitas model to disk
# torch.save(model.state_dict(), "state_dict_LeNet_WeightAct.pth")
# # torch.save(model, "LeNet_WeightAct.pth")

####  Convert to ONNX model

In [19]:
import torch
import onnx
from finn.util.test import get_test_model_trained
from brevitas.export import export_qonnx
from qonnx.util.cleanup import cleanup as qonnx_cleanup
from qonnx.core.modelwrapper import ModelWrapper
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.fold_constants import FoldConstants
from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs

# # cnv = get_test_model_trained("CNV", 1, 1)
# model_path = "state_dict_LeNet_WeightAct.pth"
# state_dict = torch.load(model_path)  # This returns an OrderedDict
# model.load_state_dict(state_dict)  # Load the state dict into the model
# # model = torch.load(model_path)

export_onnx_path = "LiDAR_WeightActBias.onnx"
# export_qonnx(model, torch.randn(1, 3, 32, 32), export_onnx_path)
export_qonnx(model, torch.randn(1, 20, 20, 20), export_onnx_path, opset_version=9) # opset_version=9 is added as ONNX version 14 is not supported.
qonnx_cleanup(export_onnx_path, out_file=export_onnx_path)

model = ModelWrapper(export_onnx_path)
model = model.transform(ConvertQONNXtoFINN())
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(RemoveStaticGraphInputs())
model.save("LiDAR_WeightActBias_tidy.onnx")

In [20]:
from finn.util.visualization import showInNetron
showInNetron("LiDAR_WeightActBias_tidy.onnx")

Serving 'LiDAR_WeightActBias_tidy.onnx' at http://0.0.0.0:8081


In [21]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
import shutil
from qonnx.core.modelwrapper import ModelWrapper

from finn.util.pytorch import ToTensor
from qonnx.transformation.merge_onnx_models import MergeONNXModels
from qonnx.core.datatype import DataType

from qonnx.transformation.insert_topk import InsertTopK
from qonnx.transformation.infer_datatypes import InferDataTypes

def custom_step_add_pre_proc(model: ModelWrapper, cfg: build.DataflowBuildConfig):
    model = ModelWrapper("LiDAR_WeightActBias_tidy.onnx")
    global_inp_name = model.graph.input[0].name
    ishape = model.get_tensor_shape(global_inp_name)
    # print('ishape = ', ishape)
    
    # preprocessing: torchvision's ToTensor divides uint8 inputs by 255
    totensor_pyt = ToTensor()
    chkpt_preproc_name = "LiDAR_WeightActBias_tidy_preproc.onnx"
    export_qonnx(totensor_pyt, torch.randn(ishape), chkpt_preproc_name, opset_version=9)
    qonnx_cleanup(chkpt_preproc_name, out_file=chkpt_preproc_name)
    pre_model = ModelWrapper(chkpt_preproc_name)
    pre_model = pre_model.transform(ConvertQONNXtoFINN())
    
    # join preprocessing and core model
    model = model.transform(MergeONNXModels(pre_model))
    
    # add input quantization annotation: UINT8 for all BNN-PYNQ models
    global_inp_name = model.graph.input[0].name
    model.set_tensor_datatype(global_inp_name, DataType["UINT8"])
    return model

def custom_step_add_post_proc(model: ModelWrapper, cfg: build.DataflowBuildConfig):
    model = model.transform(InsertTopK(k=1))
    chkpt_name = "LiDAR_WeightActBias_tidy_prepost.onnx"
    # tidy-up again
    model = model.transform(InferShapes())
    model = model.transform(FoldConstants())
    model = model.transform(GiveUniqueNodeNames())
    model = model.transform(GiveReadableTensorNames())
    model = model.transform(InferDataTypes())
    model = model.transform(RemoveStaticGraphInputs())
    model.save(chkpt_name)
    return model

# postprocessing: insert Top-1 node at the end


### Folding

In [4]:
# from finn.transformation.fpgadataflow.make_zynq_proj import ZynqBuild
# model = ModelWrapper("LeNet_WeightAct_dataflow_model.onnx")
# fc_layers = model.get_nodes_by_op_type("MVAU_hls")
# # print(fc_layers)
# # each tuple is (PE, SIMD, in_fifo_depth) for a layer
# folding = [
#     (6, 3, [16]),   # conv1
#     (8, 3, [16]),   # conv2
#     (60, 64, [64]), # fc1
#     (42, 30, [64]), # fc2
#     # (5, 42, [10]),   #fc3
# ]

# # Apply folding parameters to fully connected layers
# for fcl, (pe, simd, ififodepth) in zip(fc_layers, folding):
#     fcl_inst = getCustomOp(fcl)
#     fcl_inst.set_nodeattr("PE", pe)
#     fcl_inst.set_nodeattr("SIMD", simd)
#     fcl_inst.set_nodeattr("inFIFODepths", ififodepth)
#     # Debug: Verify parameters are set correctly
#     print(f"{fcl.name} -> PE: {pe}, SIMD: {simd}, inFIFOdepths: {ififodepth}")

# # use same SIMD values for the sliding window operators
# swg_layers = model.get_nodes_by_op_type("ConvolutionInputGenerator_rtl")

# for i, swg in enumerate(swg_layers):
#   if i < len(folding):
#       swg_inst = getCustomOp(swg)
#       simd = folding[i][1]
#       swg_inst.set_nodeattr("SIMD", simd)
#       # Debug: Verify SIMD is set correctly
#       print(f"{swg.name} -> SIMD: {simd}")
#   else:
#       print(f"No folding parameters available for {swg.name}")
      
# # Save the transformed model
# model = model.transform(GiveUniqueNodeNames())
# model.save("LeNet_WeightAct_folded.onnx")
  
# # Debug: Print the shapes to ensure compatibility
# for node in model.graph.node:
#   node_inst = getCustomOp(node)
#   try:
#       print(f"Checking folded input shape for node: {node.name}")
#       folded_shape = node_inst.get_folded_input_shape()
#       print(f"{node.name}: Folded input shape = {folded_shape}")
#   except Exception as e:
#       print(f"Error getting folded input shape for {node.name}: {e}")


In [75]:
# print(swg_layers)

In [22]:
showInNetron("LiDAR_WeightActBias_folded.onnx")

FileNotFoundError: [Errno 2] No such file or directory: 'LiDAR_WeightActBias_folded.onnx'

In [5]:
# from finn.transformation.fpgadataflow.make_zynq_proj import ZynqBuild
# test_pynq_board = "Pynq-Z1"
# target_clk_ns = 10
# model = ModelWrapper("LeNet_WeightAct_folded.onnx")
# # model = model.transform(ZynqBuild(platform = pynq_board, period_ns = target_clk_ns))
# try:
#   model = model.transform(ZynqBuild(platform=test_pynq_board, period_ns=target_clk_ns))
#   print("Hardware generation successful")
# except Exception as e:
#   print(f"Error during hardware generation: {e}")

## Understanding the Build Configuration: `DataflowBuildConfig` <a id="underst_build_conf"></a>

The build configuration is specified by an instance of `finn.builder.build_dataflow_config.DataflowBuildConfig`. The configuration is a Python [`dataclass`](https://docs.python.org/3/library/dataclasses.html) which can be serialized into or de-serialized from JSON files for persistence, although we'll just set it up in Python here.
There are many options in the configuration to customize different aspects of the build, we'll only cover a few of them in this notebook. You can read the details on all the config options on [the FINN API documentation](https://finn-dev.readthedocs.io/en/latest/source_code/finn.builder.html#finn.builder.build_dataflow_config.DataflowBuildConfig).

Let's go over some of the members of the `DataflowBuildConfig`:

### Output Products <a id="output_prod"></a>

The build can produce many different outputs, and some of them can take a long time (e.g. bitfile synthesis for a large network). When you first start working on generating a new accelerator and exploring the different performance options, you may not want to go all the way to a bitfile. Thus, in the beginning you may just select the estimate reports as the output products. Gradually, you can generate the output products from later stages until you are happy enough with the design to build the full accelerator integrated into a shell.

The output products are controlled by:

* `generate_outputs`: list of output products (of type [`finn.builder.build_dataflow_config.DataflowOutputType`](https://finn-dev.readthedocs.io/en/latest/source_code/finn.builder.html#finn.builder.build_dataflow_config.DataflowOutputType)) that will be generated by the build. Some available options are:
    - `ESTIMATE_REPORTS` : report expected resources and performance per layer and for the whole network without any synthesis
    - `STITCHED_IP` : create a stream-in stream-out IP design that can be integrated into other Vivado IPI or RTL designs
    - `RTLSIM_PERFORMANCE` : use PyVerilator to do a performance/latency test of the `STITCHED_IP` design
    - `OOC_SYNTH` : run out-of-context synthesis (just the accelerator itself, without any system surrounding it) on the `STITCHED_IP` design to get post-synthesis FPGA resources and achievable clock frequency
    - `BITFILE` : integrate the accelerator into a shell to produce a standalone bitfile
    - `PYNQ_DRIVER` : generate a PYNQ Python driver that can be used to launch the accelerator
    - `DEPLOYMENT_PACKAGE` : create a folder with the `BITFILE` and `PYNQ_DRIVER` outputs, ready to be copied to the target FPGA platform.
* `output_dir`: the directory where all the generated build outputs above will be written into.
* `steps`: list of predefined (or custom) build steps FINN will go through. Use `build_dataflow_config.estimate_only_dataflow_steps` to execute only the steps needed for estimation (without any synthesis), and the `build_dataflow_config.default_build_dataflow_steps` otherwise (which is the default value). You can find the list of default steps [here](https://finn.readthedocs.io/en/latest/source_code/finn.builder.html#finn.builder.build_dataflow_config.default_build_dataflow_steps) in the documentation.

### Configuring the Board and FPGA Part <a id="config_fpga"></a>

* `fpga_part`: Xilinx FPGA part to be used for synthesis, can be left unspecified to be inferred from `board` below, or specified explicitly for e.g. out-of-context synthesis.
* `board`: target Xilinx Zynq or Alveo board for generating accelerators integrated into a shell. See the `pynq_part_map` and `alveo_part_map` dicts in [this file](https://github.com/Xilinx/finn-base/blob/dev/src/finn/util/basic.py#L41) for a list of possible boards.
* `shell_flow_type`: the target [shell flow type](https://finn-dev.readthedocs.io/en/latest/source_code/finn.builder.html#finn.builder.build_dataflow_config.ShellFlowType), only needed for generating full bitfiles where the FINN design is integrated into a shell (so only needed if `BITFILE` is selected) 

### Configuring the Performance <a id="config_perf"></a>

You can configure the performance (and correspondingly, the FPGA resource footprint) of the generated dataflow accelerator in two ways:

1) (basic) Set a target performance and let the compiler figure out the per-node parallelization settings.

2) (advanced) Specify a separate .json as `folding_config_file` that lists the degree of parallelization (as well as other hardware options) for each layer.

This notebook only deals with the basic approach, for which you need to set up:

* `target_fps`: target inference performance in frames per second. Note that target may not be achievable due to specific layer constraints, or due to resource limitations of the FPGA. 
* `synth_clk_period_ns`: target clock frequency (in nanoseconds) for Vivado synthesis. e.g. `synth_clk_period_ns=5.0` will target a 200 MHz clock. Note that the target clock period may not be achievable depending on the FPGA part and design complexity.

## Launch a Build: Only Estimate Reports <a id="build_estimate_report"></a>

##### This is based on "cybersecurity/3-build-accelerator-with-finn.ipynb" notebook.

In [23]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
import shutil

# model_dir = os.environ['FINN_ROOT'] + "/notebooks/end2end_example/cybersecurity"
# model_file = "LeNet_WeightAct_folded.onnx"
model_file = "LiDAR_WeightActBias_tidy.onnx"
# model_file = "LeNet_WeightAct.onnx"

estimates_output_dir = "output_estimates_only"

#Delete previous run results if exist
if os.path.exists(estimates_output_dir):
    shutil.rmtree(estimates_output_dir)
    print("Previous run results deleted!")


cfg_estimates = build.DataflowBuildConfig(
    output_dir          = estimates_output_dir,
    mvau_wwidth_max     = 80,
    target_fps          = 1000000,
    synth_clk_period_ns = 10.0,
    fpga_part           = "xc7z020clg400-1",
    steps               = build_cfg.estimate_only_dataflow_steps,
    generate_outputs=[
        build_cfg.DataflowOutputType.ESTIMATE_REPORTS,
    ]
)

Previous run results deleted!


In [24]:
%%time
build.build_dataflow_cfg(model_file, cfg_estimates)

Building dataflow accelerator from LiDAR_WeightActBias_tidy.onnx
Intermediate outputs will be generated in /tmp/finn_dev_inets
Final outputs will be generated in output_estimates_only
Build log is at output_estimates_only/build_dataflow.log
Running step: step_qonnx_to_finn [1/10]
Running step: step_tidy_up [2/10]
Running step: step_streamline [3/10]
Running step: step_convert_to_hw [4/10]
Running step: step_create_dataflow_partition [5/10]
Running step: step_specialize_layers [6/10]
Running step: step_target_fps_parallelization [7/10]
Running step: step_apply_folding_config [8/10]
Running step: step_minimize_bit_width [9/10]
Running step: step_generate_estimate_reports [10/10]
Completed successfully
CPU times: user 1.14 s, sys: 3.99 ms, total: 1.15 s
Wall time: 1.16 s


0

In [25]:
import json
def read_json_dict(filename):
    with open(filename, "r") as f:
        ret = json.load(f)
    return ret

In [26]:
read_json_dict(estimates_output_dir + "/report/estimate_network_performance.json")

{'critical_path_cycles': 27840,
 'max_cycles': 6400,
 'max_cycles_node_name': 'MVAU_hls_0',
 'estimated_throughput_fps': 15625.0,
 'estimated_latency_ns': 278400.0}

In [1]:
# read_json_dict(estimates_output_dir + "/report/estimate_layer_cycles.json")

In [2]:
# read_json_dict(estimates_output_dir + "/report/op_and_param_counts.json")

In [3]:
# read_json_dict(estimates_output_dir + "/report/estimate_layer_resources.json")

In [34]:
# read_json_dict(estimates_output_dir + "/report/estimate_layer_config_alternatives.json")

## Launch a Build: Stitched IP, out-of-context synth and rtlsim Performance <a id="build_ip_synth_rtlsim"></a>

Once we have a configuration that gives satisfactory estimates, we can move on to generating the accelerator. We can do this in different ways depending on how we want to integrate the accelerator into a larger system. For instance, if we have a larger streaming system built in Vivado or if we'd like to re-use this generated accelerator as an IP component in other projects, the `STITCHED_IP` output product is a good choice. We can also use the `OOC_SYNTH` output product to get post-synthesis resource and clock frequency numbers for our accelerator.

<font color="red">**Live FINN tutorial:** These next builds will take about 10 minutes to complete since multiple calls to Vivado and a call to RTL simulation are involved. While this is running, you can examine the generated files with noVNC -- it is running on **(your AWS URL):6080/vnc.html**

* Once the `step_hls_codegen [8/16]` below is completed, you can view the generated HLS code under its own folder for each layer: `/tmp/finn_dev_ubuntu/code_gen_ipgen_MVAU_hls_XXXXXX`
    
* Once the `step_create_stitched_ip [11/16]` below is completed, you can view the generated stitched IP in Vivado under `/home/ubuntu/finn/notebooks/end2end_example/cybersecurity/output_ipstitch_ooc_rtlsim/stitched_ip`
</font> 

In [27]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import finn.builder.build_dataflow_steps as build_steps
import os
import shutil

model_file = "LiDAR_WeightActBias_tidy.onnx"
# model_file = "LeNet_WeightAct.onnx"
# model_file = "LeNet_WeightAct_folded.onnx"

rtlsim_output_dir = "output_ipstitch_ooc_rtlsim"

#Delete previous run results if exist
if os.path.exists(rtlsim_output_dir):
    shutil.rmtree(rtlsim_output_dir)
    print("Previous run results deleted!")

cfg_stitched_ip = build.DataflowBuildConfig(
    output_dir          = rtlsim_output_dir,
    mvau_wwidth_max     = 10,   # 10, 15 works, 20 not. (mvau: matrix vector activation unit)
    target_fps          = 1000000,#1000000,
    synth_clk_period_ns = 10.0,
    fpga_part           = "xc7z020clg400-1",
    # specialize_layers_config_file="specialize_layers_config.json",
    steps = [custom_step_add_pre_proc, custom_step_add_post_proc] + build_cfg.default_build_dataflow_steps,
    # steps = [ build_steps.step_hw_codegen, build_steps.step_hw_ipgen, build_steps.step_set_fifo_depths,build_steps.step_create_stitched_ip], #, '', ''step_create_stitched_ip, 'step_measure_rtlsim_performance'
    generate_outputs=[
        build_cfg.DataflowOutputType.ESTIMATE_REPORTS,
        build_cfg.DataflowOutputType.STITCHED_IP,
        build_cfg.DataflowOutputType.RTLSIM_PERFORMANCE,
        # build_cfg.DataflowOutputType.OOC_SYNTH,
    ]
)

# Next step: try to figure out in which file the folding is done and edit that file to apply folding manually.


Previous run results deleted!


In [28]:
%%time
build.build_dataflow_cfg(model_file, cfg_stitched_ip)

Building dataflow accelerator from LiDAR_WeightActBias_tidy.onnx
Intermediate outputs will be generated in /tmp/finn_dev_inets
Final outputs will be generated in output_ipstitch_ooc_rtlsim
Build log is at output_ipstitch_ooc_rtlsim/build_dataflow.log
Running step: custom_step_add_pre_proc [1/21]
Running step: custom_step_add_post_proc [2/21]
Running step: step_qonnx_to_finn [3/21]
Running step: step_tidy_up [4/21]
Running step: step_streamline [5/21]
Running step: step_convert_to_hw [6/21]
Running step: step_create_dataflow_partition [7/21]
Running step: step_specialize_layers [8/21]
Running step: step_target_fps_parallelization [9/21]
Running step: step_apply_folding_config [10/21]
Running step: step_minimize_bit_width [11/21]
Running step: step_generate_estimate_reports [12/21]
Running step: step_hw_codegen [13/21]
Running step: step_hw_ipgen [14/21]
Running step: step_set_fifo_depths [15/21]
Running step: step_create_stitched_ip [16/21]
Running step: step_measure_rtlsim_performance 

0

In [29]:
import json
def read_json_dict(filename):
    with open(filename, "r") as f:
        ret = json.load(f)
    return ret

In [30]:
read_json_dict(rtlsim_output_dir + "/report/estimate_network_performance.json")

{'critical_path_cycles': 192007,
 'max_cycles': 64000,
 'max_cycles_node_name': 'MVAU_hls_0',
 'estimated_throughput_fps': 1562.5,
 'estimated_latency_ns': 1920070.0}

In [31]:
assert os.path.exists(rtlsim_output_dir + "/report/ooc_synth_and_timing.json")
assert os.path.exists(rtlsim_output_dir + "/report/rtlsim_performance.json")
assert os.path.exists(rtlsim_output_dir + "/final_hw_config.json")

AssertionError: 

In [32]:
! ls {rtlsim_output_dir}/stitched_ip

all_verilog_srcs.txt		       finn_vivado_stitch_proj.xpr
data				       ip
finn_vivado_stitch_proj.cache	       make_project.sh
finn_vivado_stitch_proj.gen	       make_project.tcl
finn_vivado_stitch_proj.hw	       vivado.jou
finn_vivado_stitch_proj.ip_user_files  vivado.log
finn_vivado_stitch_proj.srcs


In [33]:
! ls {rtlsim_output_dir}/report

estimate_layer_config_alternatives.json  estimate_network_performance.json
estimate_layer_cycles.json		 op_and_param_counts.json
estimate_layer_resources_hls.json	 rtlsim_performance.json
estimate_layer_resources.json


In [34]:
! cat {rtlsim_output_dir}/report/ooc_synth_and_timing.json

cat: output_ipstitch_ooc_rtlsim/report/ooc_synth_and_timing.json: No such file or directory


In [35]:
! cat {rtlsim_output_dir}/report/rtlsim_performance.json

{
  "N_IN_TXNS": 8000,
  "N_OUT_TXNS": 1,
  "cycles": 125536,
  "N": 1,
  "latency_cycles": 125536,
  "runtime[ms]": 1.25536,
  "throughput[images/s]": 796.5842467499363,
  "fclk[mhz]": 100.0,
  "stable_throughput[images/s]": 796.5842467499363
}

In [36]:
! cat {rtlsim_output_dir}/final_hw_config.json

{
  "Defaults": {},
  "StreamingFIFO_rtl_0": {
    "ram_style": "auto",
    "depth": 2,
    "impl_style": "rtl",
    "inFIFODepths": [
      0
    ],
    "outFIFODepths": [
      0
    ]
  },
  "Thresholding_rtl_0": {
    "PE": 1,
    "runtime_writeable_weights": 0,
    "inFIFODepths": [
      2
    ],
    "outFIFODepths": [
      3415
    ],
    "depth_trigger_uram": 0,
    "depth_trigger_bram": 0
  },
  "StreamingFIFO_rtl_1": {
    "ram_style": "auto",
    "depth": 3415,
    "impl_style": "vivado",
    "inFIFODepths": [
      0
    ],
    "outFIFODepths": [
      0
    ]
  },
  "StreamingDataWidthConverter_rtl_0": {
    "inFIFODepths": [
      3415
    ],
    "outFIFODepths": [
      2000
    ]
  },
  "StreamingFIFO_rtl_2": {
    "ram_style": "auto",
    "depth": 2000,
    "impl_style": "vivado",
    "inFIFODepths": [
      0
    ],
    "outFIFODepths": [
      0
    ]
  },
  "ConvolutionInputGenerator_rtl_0": {
    "SIMD": 4,
    "parallel_window": 0,
    "ram_style": "distributed",